# 사이킷런의 LabelBinarizer를 사용하여 문자열 타깃 데이터 원-핫 인코딩

## 싱글 클래스 특성에 대한 처리 방법

In [2]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [3]:
feature = np.array((["가나다라"],
                   ["가나다라"],
                   ["아바하자"],
                   ["카나다사"]))
one_hot = LabelBinarizer()  # 원-핫 인코더 생성
one_hot.fit_transform(feature)  # 특성을 원-핫 인코딩 변환
one_hot.classes_  # 특성 클래스 확인

print("특성 클래스 확인 ", one_hot.classes_)

특성 클래스 확인  ['가나다라' '아바하자' '카나다사']


In [4]:
# 데이터를 되돌리는 방법
one_hot_data = one_hot.inverse_transform(one_hot.transform(feature))
print("one hot >> ", one_hot_data)

one hot >>  ['가나다라' '가나다라' '아바하자' '카나다사']


## 다중 클래스 특성에 대한 처리 방법

In [5]:
multiclass_feature = [("가나다라마", "아자바하나"),
                     ("자다가나라", "자다나타하"),
                     ("가나다라마", "아자바하나"),
                     ("아마자나가", "아카나다하"),
                     ("가나다라마", "아자바하나")]
one_hot_multiclass = MultiLabelBinarizer()  # 다중 클래스 원-핫 인코더 객체 생성
one_hot_multiclass.fit_transform(multiclass_feature)  # 다중 클래스 특성을 원-핫 인코딩 변환
one_hot_data = one_hot_multiclass.classes_  # 특성 클래스 확인
print("특성 클래스 확인 >> ", one_hot_data)

특성 클래스 확인 >>  ['가나다라마' '아마자나가' '아자바하나' '아카나다하' '자다가나라' '자다나타하']


## 문자열 타깃 데이터 원-핫 인코딩

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
# 여러 개 열이 있는 특성 배열 생성
str_feature = np.array([["안녕", 1],
                       ["저녁", 3],
                       ["안녕", 1],
                       ["점심", 1]])

In [8]:
# 희소배열로 반환이 기본값이며 sparse=False로 지정하면 밀집배열로 전환
one_hot_encoder = OneHotEncoder(sparse=False)

In [9]:
# OneHotencoder -> 입력 특성 배열을 모두 범주형으로 인식
one_hot_encoder.fit_transform(str_feature)
one_hot_encoder_data = one_hot_encoder.categories_  # categories_ 속성으로 클래스를 확인 가능
print("one hot encoder data >> ", one_hot_encoder_data)

one hot encoder data >>  [array(['안녕', '저녁', '점심'], dtype='<U11'), array(['1', '3'], dtype='<U11')]


C:\Users\bomi\anaconda3\envs\AI\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 순서가 있는 범주형 특성 인코딩

In [10]:
# 판다스 데이터프레임의 replace()를 사용하여 문자열 레이블을 수치값으로 변환합니다.
import pandas as pd

In [11]:
# 특성 데이터 생성
dataframe = pd.DataFrame({'Score' : ["Low", "Low", "Medium", "Medium", "High"]})

In [13]:
# 매핑 딕셔너리 생성
scale_mapper = {"Low" : 1,
               "Medium" : 2,
               "High" : 3}

data = dataframe["Score"].replace(scale_mapper)  # 특성을 정수로 변환
print(data)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64


## 순서가 있는 범주형 특성 인코딩 01

In [14]:
# 범주형 변수를 카테고리 값이 0부터 시작하는 정수 인덱스로 변환
from sklearn.preprocessing import OrdinalEncoder

In [15]:
features_array = np.array([["Low", 10],
                          ["High", 50],
                          ["Medium", 3]])

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(features_array)
ordinal_encoder_data = ordinal_encoder.categories_

print("Ordinal Encoder >> ", ordinal_encoder_data)

Ordinal Encoder >>  [array(['High', 'Low', 'Medium'], dtype='<U11'), array(['10', '3', '50'], dtype='<U11')]


## 순서가 있는 범주형 특성 인코딩 02

In [17]:
# 특성 딕셔너리 인코딩
from sklearn.feature_extraction import DictVectorizer

In [22]:
# 문서에서 단어의 사용 빈도를 나타내는 딕셔너리 정보를 입력받아 BOW 인코딩한 수치 벡터로 변환한다.
# 딕셔너리 생성
data_dict = [{"Red" : 2, "Blue" : 4},
            {"Red" : 4, "Blue" : 3},
            {"Red" : 1, "Yellow" : 2},
            {"Red" : 1, "Yellow" : 2}]

In [23]:
# DictVectorizer 객체 생성
dictvectorizer = DictVectorizer(sparse=False)

features = dictvectorizer.fit_transform(data_dict)  # 데이터를 특성 행렬로 변환
print(features)

[[4. 2. 0.]
 [3. 4. 0.]
 [0. 1. 2.]
 [0. 1. 2.]]


In [25]:
feature_names = dictvectorizer.get_feature_names_out()
print(feature_names)

['Blue' 'Red' 'Yellow']


In [26]:
dict_data = pd.DataFrame(features, columns=feature_names)
print(dict_data)

   Blue  Red  Yellow
0   4.0  2.0     0.0
1   3.0  4.0     0.0
2   0.0  1.0     2.0
3   0.0  1.0     2.0


# 누락된 클래스 값 대처하기 01

## 머신러닝 분류 알고리즘을 훈련 -> 누락된 값 예측 (k-최근접 이웃 KNN 사용)

In [27]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [28]:
# 범주형 특성을 가진 특성 행렬 생성
x=np.array([[0, 2.10, 1.45],
           [1, 1.18, 1.33],
           [0, 1.22, 1.27],
           [1, -0.20, -1.15]])

In [31]:
# 범주형 특성에 누락된 값이 있는 특성 행렬 생성
x_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])
clf = KNeighborsClassifier(3, weights='distance')  # k = 3, 가장 가까운 이웃을 찾아 거리에 가중치를 부여하여 예측

print(x[:,1:])  # 데이터
print(x[:,0])  # 라벨

[[ 2.1   1.45]
 [ 1.18  1.33]
 [ 1.22  1.27]
 [-0.2  -1.15]]
[0. 1. 0. 1.]


In [32]:
train_model = clf.fit(x[:,1:], x[:,0])  # 훈련
imputed_values = train_model.predict(x_with_nan[:,1:])  # 누락된 값의 클래스 예측

In [33]:
# 예측된 클래스와 원본 특성을열로 합칩니다.
x_with_imputed = np.hstack((imputed_values.reshape(-1,1), x_with_nan[:,1:]))
data = np.vstack((x_with_imputed, x))  # 두 특성 행렬을 연결

print(data)

[[ 0.    0.87  1.31]
 [ 1.   -0.67 -0.22]
 [ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.2  -1.15]]


# 누락된 클래스 값 대처하기 02

## 가장 자주 등장하는 값으로 채우기

In [34]:
# 두 개의 특성 행렬을 합칩니다.
from sklearn.impute import SimpleImputer  # 결측값을 대체

strategy 들어갈 수 있는 값은 총 3개<br>
mean: 결측값을 해당 특성의 평균값으로 대체합니다.<br>
median: 결측값을 해당 특성의 중앙값으로 대체합니다.<br>
most_frequent: 결측값을 해당 특성에서 가장 자주 등장하는 값으로 대체합니다.
    
이러한 전략 중 어떤 것을 선택할지는 데이터의 특성과 상황에 따라 다릅니다.
일반적으로는 수치형 데이터의 경우 mean이나 median을, 범주형 데이터의 경우 most_frequent를 사용합니다.

In [35]:
x_complete = np.vstack((x_with_nan, x))

imputer = SimpleImputer(strategy = "most_frequent")
data_imputer = imputer.fit_transform(x_complete)
print(data_imputer)

[[ 0.    0.87  1.31]
 [ 0.   -0.67 -0.22]
 [ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.2  -1.15]]
